In [66]:
#import necessary modules
import pandas as pd
pd.options.display.max_columns=1000
pd.options.display.width=200
pd.options.display.min_rows=60

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing 
from sklearn.feature_extraction.text import TfidfVectorizer

In [67]:
#read in cleaned data file
df =  pd.read_csv('data_cleaned4.csv')
df.head()

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDISDBINCL,CLNDN,CLNDNINCL,CLNHGVS,CLNSIGINCL,CLNVI,MC,ORIGIN,SSR,CLASS,Allele,Consequence,IMPACT,SYMBOL,Feature_type,Feature,EXON,INTRON,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,DISTANCE,STRAND,BAM_EDIT,PolyPhen,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62,CHROM_1,CHROM_2,CHROM_3,CHROM_4,CHROM_5,CHROM_6,CHROM_7,CHROM_8,CHROM_9,CHROM_10,CHROM_11,CHROM_12,CHROM_13,CHROM_14,CHROM_15,CHROM_16,CHROM_16.1,CHROM_17,CHROM_18,CHROM_19,CHROM_20,CHROM_21,CHROM_22,CHROM_MT,CHROM_X,CLNVC_Deletion,CLNVC_Duplication,CLNVC_Indel,CLNVC_Insertion,CLNVC_Inversion,CLNVC_Microsatellite,CLNVC_single_nucleotide_variant,INT,EX,SIFT_deleterious,SIFT_deleterious_low_confidence,SIFT_tolerated,SIFT_tolerated_low_confidence,SIFT_unknown,BIOTYPE_misc_RNA,BIOTYPE_protein_coding,BIOTYPE_unknown,Length,Relative_Location
0,955563,G,C,0.0000,0.00000,0.0000,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",NaN,NC_000001.10:g.955563G>C,NaN,NaN,SO:0001583|missense_variant,1.0,0.0,0,C,missense_variant,MODERATE,AGRN,Transcript,NM_001305275.1,1/39,NaN,61,11,4.0,R/P,cGg/cCg,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.421,11.390,1.133255,-2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,2068.0,0.001934
1,955597,G,T,0.0000,0.42418,0.2826,MedGen:CN169374,NaN,not_specified,NaN,NC_000001.10:g.955597G>T,NaN,NaN,SO:0001819|synonymous_variant,1.0,0.0,0,T,synonymous_variant,LOW,AGRN,Transcript,NM_001305275.1,1/39,NaN,95,45,15.0,P,ccG/ccT,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.421,8.150,0.599088,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,2068.0,0.007253
2,955619,G,C,0.0000,0.03475,0.0088,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",NaN,NC_000001.10:g.955619G>C,NaN,NaN,SO:0001583|missense_variant,1.0,0.0,1,C,missense_variant,MODERATE,AGRN,Transcript,NM_001305275.1,1/39,NaN,117,67,23.0,V/L,Gtc/Ctc,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.421,3.288,0.069819,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,2068.0,0.011122
3,957640,C,T,0.0318,0.02016,0.0328,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",NaN,NC_000001.10:g.957640C>T,NaN,NaN,SO:0001819|synonymous_variant,1.0,0.0,0,T,synonymous_variant,LOW,AGRN,Transcript,NM_001305275.1,2/39,NaN,311,261,87.0,D,gaC/gaT,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.421,12.560,1.356499,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,2068.0,0.042070
4,976059,C,T,0.0000,0.00022,0.0010,MedGen:CN169374,NaN,not_specified,NaN,NC_000001.10:g.976059C>T,NaN,NaN,SO:0001819|synonymous_variant,1.0,0.0,1,T,synonymous_variant,LOW,AGRN,Transcript,NM_001305275.1,4/39,NaN,576,526,176.0,L,Ctg/Ttg,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.421,17.740,2.234711,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,2068.0,0.085106


In [68]:
#create a TfidfVectorizer object, with tokenizer=list for the gene sequence strings
vectorizer = TfidfVectorizer(tokenizer=list)

In [69]:
#perform a .fit_transform on the "REF" column (reference allele)
X_REF = vectorizer.fit_transform(df.REF)

#print the feature names; should be the four nucleotide bases
print(vectorizer.get_feature_names())

#print the shape of the resultant csr sparse matrix
print(X_REF.shape)

['a', 'c', 'g', 't']
(65188, 4)


In [70]:
#perform the same steps as above on the "ALT" column, that is, the alternate allele
X_ALT = vectorizer.fit_transform(df.ALT)
print(vectorizer.get_feature_names())
print(X_ALT.shape)

['a', 'c', 'g', 't']
(65188, 4)


In [71]:
#print the type of the vectorizer and X_ALT for clarity
print(type(vectorizer))
print(type(X_ALT))

<class 'sklearn.feature_extraction.text.TfidfVectorizer'>
<class 'scipy.sparse.csr.csr_matrix'>


In [72]:
#Convert the sparse matrix to a dense type, then use to create a data frame with "ALT_" & "REF_" column name prefixes
tfidf_ALT = pd.DataFrame(X_ALT.todense(),columns=['ALT_' + x for x in vectorizer.get_feature_names()])
tfidf_ALT.head()
tfidf_REF = pd.DataFrame(X_REF.todense(),columns=['REF_' + x for x in vectorizer.get_feature_names()])

In [73]:
#Check an entry where there is more than one nucleotide represented (i.e. the sum across all four bases does not equal zero)
list(tfidf_ALT.T.sum() > 1).index(True)

134

In [74]:
#View the ALT allele values in the sparse matrix 
tfidf_ALT.iloc[134]

ALT_a    0.000000
ALT_c    0.784158
ALT_g    0.000000
ALT_t    0.620561
Name: 134, dtype: float64

In [75]:
#Confirm the distribution created by TfidfVectorizer matches with the actual ALT allele = "CT" -- it does!
df.iloc[134]

POS                                                                          2338230
REF                                                                                C
ALT                                                                               CT
AF_ESP                                                                             0
AF_EXAC                                                                            0
AF_TGP                                                                             0
CLNDISDB                           MedGen:C3553947,OMIM:614870|MedGen:C3553948,OM...
CLNDISDBINCL                                                                     NaN
CLNDN                              Peroxisome_biogenesis_disorder_6A|Peroxisome_b...
CLNDNINCL                                                                        NaN
CLNHGVS                                                   NC_000001.10:g.2338231dupT
CLNSIGINCL                                                       

In [76]:
#Concatenate the newly engineered 8 features (4 from REF and 4 from ALT into the original dataframe)
df_nlp2 = pd.concat([df,tfidf_REF,tfidf_ALT], axis=1, sort=False)

In [77]:
df_nlp2.head()

,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDISDBINCL,CLNDN,CLNDNINCL,CLNHGVS,CLNSIGINCL,CLNVI,MC,ORIGIN,SSR,CLASS,Allele,Consequence,IMPACT,SYMBOL,Feature_type,Feature,EXON,INTRON,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,DISTANCE,STRAND,BAM_EDIT,PolyPhen,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62,CHROM_1,CHROM_2,CHROM_3,CHROM_4,CHROM_5,CHROM_6,CHROM_7,CHROM_8,CHROM_9,CHROM_10,CHROM_11,CHROM_12,CHROM_13,CHROM_14,CHROM_15,CHROM_16,CHROM_16.1,CHROM_17,CHROM_18,CHROM_19,CHROM_20,CHROM_21,CHROM_22,CHROM_MT,CHROM_X,CLNVC_Deletion,CLNVC_Duplication,CLNVC_Indel,CLNVC_Insertion,CLNVC_Inversion,CLNVC_Microsatellite,CLNVC_single_nucleotide_variant,INT,EX,SIFT_deleterious,SIFT_deleterious_low_confidence,SIFT_tolerated,SIFT_tolerated_low_confidence,SIFT_unknown,BIOTYPE_misc_RNA,BIOTYPE_protein_coding,BIOTYPE_unknown,Length,Relative_Location,REF_a,REF_c,REF_g,REF_t,ALT_a,ALT_c,ALT_g,ALT_t
0,955563,G,C,0.0000,0.00000,0.0000,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",NaN,NC_000001.10:g.955563G>C,NaN,NaN,SO:0001583|missense_variant,1.0,0.0,0,C,missense_variant,MODERATE,AGRN,Transcript,NM_001305275.1,1/39,NaN,61,11,4.0,R/P,cGg/cCg,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.421,11.390,1.133255,-2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,2068.0,0.001934,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,955597,G,T,0.0000,0.42418,0.2826,MedGen:CN169374,NaN,not_specified,NaN,NC_000001.10:g.955597G>T,NaN,NaN,SO:0001819|synonymous_variant,1.0,0.0,0,T,synonymous_variant,LOW,AGRN,Transcript,NM_001305275.1,1/39,NaN,95,45,15.0,P,ccG/ccT,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.421,8.150,0.599088,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,2068.0,0.007253,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,955619,G,C,0.0000,0.03475,0.0088,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",NaN,NC_000001.10:g.955619G>C,NaN,NaN,SO:0001583|missense_variant,1.0,0.0,1,C,missense_variant,MODERATE,AGRN,Transcript,NM_001305275.1,1/39,NaN,117,67,23.0,V/L,Gtc/Ctc,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.421,3.288,0.069819,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,2068.0,0.011122,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,957640,C,T,0.0318,0.02016,0.0328,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",NaN,NC_000001.10:g.957640C>T,NaN,NaN,SO:0001819|synonymous_variant,1.0,0.0,0,T,synonymous_variant,LOW,AGRN,Transcript,NM_001305275.1,2/39,NaN,311,261,87.0,D,gaC/gaT,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.421,12.560,1.356499,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,2068.0,0.042070,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,976059,C,T,0.0000,0.00022,0.0010,MedGen:CN169374,NaN,not_specified,NaN,NC_000001.10:g.976059C>T,NaN,NaN,SO:0001819|synonymous_variant,1.0,0.0,1,T,synonymous_variant,LOW,AGRN,Transcript,NM_001305275.1,4/39,NaN,576,526,176.0,L,Ctg/Ttg,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.421,17.740,2.234711,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,2068.0,0.085106,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [78]:
#Save the dataframe in a csv file
df_nlp2.to_csv('data_cleanednlp4.csv', index=False)